In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import pandas as pd

In [2]:

# import deep learning LSTM libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import r2_score
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping


2024-05-23 19:27:35.334760: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 19:27:35.441097: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 19:27:35.947700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 19:27:37.390885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv('data/data-ready.csv')

df = df.drop("DateTime", axis = 1)

df = df.drop(["Month", "Day", "Hour"], axis=1)

df_z1 = df.drop(["z2", "z3"], axis=1)
df_z2 = df.drop(["z1", "z3"], axis=1)
df_z3 = df.drop(["z1", "z2"], axis=1)

df_total = df.copy()
df_total["z_total"] = df_total["z1"] + df_total["z2"] + df_total["z3"]

# drop z1, z2, z3 from df_total
df_total = df_total.drop(["z1", "z2", "z3"], axis=1)

df.head()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,z1,z2,z3,Year,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
0,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386,2017,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
1,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434,2017,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
2,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373,2017,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
3,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711,2017,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299
4,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964,2017,1,1,1.0,0.0,0.866025,0.5,0.97953,0.201299


In [4]:
zones = [df_z1, df_z2, df_z3, df_total]
targets = ["z1", "z2", "z3", "z_total"]

In [5]:
# Reshape the data for the LSTM model
def create_sequences(data, target, k):
    X, y = [], []
    for i in range(k, len(data)):
        X.append(data[i-k:i])
        y.append(target[i])
    return np.array(X), np.array(y)

In [6]:
from sklearn.preprocessing import StandardScaler

time_steps = 24

zone = df_z1
target = "z1"

# for each zone, train LSTM model
for zone, target in zip(zones, targets):
    print(f"==================== {target} ====================")

    X = zone.drop(target, axis=1)
    y = zone[target]


    # Feature Scaling
    sc = MinMaxScaler()
    X_scaled = sc.fit_transform(X)

    # Standardize the target
    sc_y = StandardScaler()
    y = y.values.reshape(-1, 1)
    y_scaled = sc_y.fit_transform(y)


    X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)


    # Initialising the RNN
    model = Sequential()

    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))

    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_split=0.2)


    y_pred = model.predict(X_test)


    # Inverse transform predictions and actual values if you want to interpret in original scale
    y_pred_inv = sc_y.inverse_transform(y_pred)
    y_test_inv = sc_y.inverse_transform(y_test.reshape(-1, 1))

    # Calculate some performance metrics
    mse = mean_squared_error(y_test_inv, y_pred_inv)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    
    print(f'Mean Squared Error for {target}: {mse}')
    print(f'Mean Absolute Error for {target}: {mae}')
    print(f'R² Score for {target}: {r2}')


==================== z1 ====================


/home/froggy/miniforge3/envs/DM_ENV/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.2462 - val_loss: 0.0821
Epoch 2/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0929 - val_loss: 0.0789
Epoch 3/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0811 - val_loss: 0.0643
Epoch 4/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0702 - val_loss: 0.0616
Epoch 5/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - loss: 0.0656 - val_loss: 0.0554
Epoch 6/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 0.0604 - val_loss: 0.0516
Epoch 7/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0549 - val_loss: 0.0458
Epoch 8/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - loss: 0.0500 - val_loss: 0.0430
Epoch 9/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0486 - val_loss: 0.0422
Epoch 10/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0452 - val_loss: 0.0390
Epoch 11/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0429 - val_loss: 0.0466
Epoch 12

/home/froggy/miniforge3/envs/DM_ENV/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1048/1048 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - loss: 0.3014 - val_loss: 0.1084
Epoch 2/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.1144 - val_loss: 0.0808
Epoch 3/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - loss: 0.0932 - val_loss: 0.0687
Epoch 4/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0807 - val_loss: 0.0608
Epoch 5/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0714 - val_loss: 0.0625
Epoch 6/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - loss: 0.0649 - val_loss: 0.0506
Epoch 7/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0617 - val_loss: 0.0487
Epoch 8/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0570 - val_loss: 0.0480
Epoch 9/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - loss: 0.0559 - val_loss: 0.0465
Epoch 10/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0524 - val_loss: 0.0529
Epoch 11/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0504 - val_loss: 0.0444
Epoch 12/20
1048/10

/home/froggy/miniforge3/envs/DM_ENV/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1048/1048 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.2558 - val_loss: 0.0725
Epoch 2/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - loss: 0.0691 - val_loss: 0.0383
Epoch 3/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0493 - val_loss: 0.0301
Epoch 4/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - loss: 0.0408 - val_loss: 0.0268
Epoch 5/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - loss: 0.0373 - val_loss: 0.0277
Epoch 6/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - loss: 0.0352 - val_loss: 0.0368
Epoch 7/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0336 - val_loss: 0.0232
Epoch 8/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0295 - val_loss: 0.0190
Epoch 9/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - loss: 0.0274 - val_loss: 0.0177
Epoch 10/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 0.0291 - val_loss: 0.0296
Epoch 11/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0259 - val_loss: 0.0171
Epoch 12/20
1048/10

/home/froggy/miniforge3/envs/DM_ENV/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1048/1048 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 0.2028 - val_loss: 0.0740
Epoch 2/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - loss: 0.0727 - val_loss: 0.0574
Epoch 3/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - loss: 0.0607 - val_loss: 0.0409
Epoch 4/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - loss: 0.0502 - val_loss: 0.0384
Epoch 5/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0455 - val_loss: 0.0345
Epoch 6/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - loss: 0.0407 - val_loss: 0.0277
Epoch 7/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - loss: 0.0379 - val_loss: 0.0270
Epoch 8/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - loss: 0.0351 - val_loss: 0.0266
Epoch 9/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - loss: 0.0346 - val_loss: 0.0227
Epoch 10/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0317 - val_loss: 0.0217
Epoch 11/20
1048/1048 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - loss: 0.0301 - val_loss: 0.0220
Epoch 12/20
1048/10